In [3]:
# Main function for getting data and computing the model

from pandas.io import sql
import pymysql as mdb
from datetime import datetime
import random
import numpy as np
import pandas
from brickset import *
from amazonproduct import API
from bs4 import BeautifulSoup
# from html5lib import *
import lxml
# import matplotlib.pyplot
import urllib2
from bs4 import BeautifulSoup
# import matplotlib.pyplot as plt
import itertools
import time
from time import mktime
import re

In [4]:
########### DATA PANDA SETUP ############

cols = ['SetNumber','SetName','Theme','ASIN','retailPrice','currentPrice_A',
    'highPrice_A','lowPrice_A','avgPrice_A','highPrice_3','lowPrice_3','avgPrice_3',
    'introDate','introYear','selloutDate','highPriceDate_A','pieces','minifigs','rating_B',
       'want_it','have_it']

legoSets = pandas.DataFrame(data=np.zeros((0,len(cols))), columns=cols)

# get the panda ready for data
legoSets.loc[len(legoSets)] = np.nan

In [5]:
legoSets

,SetNumber,SetName,Theme,ASIN,retailPrice,currentPrice_A,highPrice_A,lowPrice_A,avgPrice_A,highPrice_3,...,avgPrice_3,introDate,introYear,selloutDate,highPriceDate_A,pieces,minifigs,rating_B,want_it,have_it
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
########## GET DATA METHODS #############

amazon_api = API(locale='us') # setup amazon API
brickset_client = brickset() # setup the brickset API client

#### ALL ITEM SEARCH
def am_search(SetName):
    # get only LEGO model number
    search_result = amazon_api.item_search('Toys', Keywords=SetName)
    try:
        for item in search_result:
            # use only the top result for now
            return item.ASIN.text
    except:
        return NaN
    
#### function call that searches & verifies the LEGO set and sets up the LEGO object
def amLookupASIN(iASIN,myPanda):
    p = re.compile(ur'[0-9]{3,10}')
    lookUP = amazon_api.item_lookup(iASIN)
    for item in lookUP.Items.Item:
        myPanda.loc[len(legoSets)-1,'ASIN']=np.squeeze(item.ASIN) # use only the top result for now
        myPanda.loc[len(legoSets)-1,'SetName']=item.ItemAttributes.Title.text
        myPanda.loc[len(legoSets)-1,'SetNumber']=re.search(p,item.ItemAttributes.Title.text).group(0)
    return myPanda

In [7]:
## test case
name = 'LEGO Star Wars Poes X-Wing Fighter'

# name = raw_input('Which LEGO set are you looking for?')
ASIN = am_search(name) # get ASIN
legoSets = amLookupASIN(ASIN,legoSets)
SetNumber=legoSets.loc[len(legoSets)-1,'SetNumber']

## Amazon Scratch Pad Code

In [8]:
legoSets

,SetNumber,SetName,Theme,ASIN,retailPrice,currentPrice_A,highPrice_A,lowPrice_A,avgPrice_A,highPrice_3,...,avgPrice_3,introDate,introYear,selloutDate,highPriceDate_A,pieces,minifigs,rating_B,want_it,have_it
0,75102,LEGO Star Wars Poe's X-Wing Fighter 75102 Buil...,NaN,B00WHZ6WOO,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## CamelCamelCamel Scraping Function

In [9]:
def getCamels(ASIN,myPanda):
    tryurl =  "http://www.camelcamelcamel.com/product/"+ASIN # setup the url
    # print tryurl
    response = urllib2.urlopen(tryurl)
    html = response.read()

    parsedHtml = BeautifulSoup(html,"lxml") 	# parse with BeautifulSoup
    # print parsedHtml

    # get introduction date for set
    items = parsedHtml.find_all(attrs={"class": "smalltext grey"})
    for item in items:
        if 'since' in item.text:
            dt = item.text.lstrip().replace("* since", "").replace('+ of the last 50 price changes','')
            myPanda.loc[len(legoSets)-1,'introDate'] = pandas.to_datetime(dt)
            break
                
    divs = parsedHtml.find_all("div", {"class":"yui3-u-1-2"}) # find the divs with the price information
    for div in divs:
        if 'Amazon' in div.h3.text: # get amazon prices
            temp_text = div.text.split('\n') # split by line 	# print div.text
            temp_text = filter(None, temp_text) # remove empty
            for I in range(0,len(temp_text)):
                if 'Highest' in temp_text[I]:
                    myPanda.loc[len(legoSets)-1,'highPrice_A'] = float(temp_text[I+1].replace("$", ""))
                    # print pandas.to_datetime(pandas.Series(temp_text[I+2].lstrip()))
                    # print temp_text[I+2]
                    # print time.strptime(temp_text[I+2].lstrip(), "%b %d, %Y")
                    myPanda.loc[len(legoSets)-1,'highPriceDate_A'] = pandas.to_datetime(temp_text[I+2].lstrip())
                    # time.strptime(temp_text[I+2].lstrip(), "%b %d, %Y")
                if 'Lowest' in temp_text[I]:
                    myPanda.loc[len(legoSets)-1,'lowPrice_A'] = float(temp_text[I+1].replace("$", ""))
                if 'Average' in temp_text[I]:
                    myPanda.loc[len(legoSets)-1,'avgPrice_A'] = float(temp_text[I+1].replace("$", ""))
                if 'Current' in temp_text[I]: # current price or not in stock
                    try:
                        myPanda.loc[len(legoSets)-1,'currentPrice_A'] = float(temp_text[I+1].replace("$", ""))
                    except ValueError:
                        myPanda.loc[len(legoSets)-1,'currentPrice_A'] = 'NaN' # means not in stock
                        temp_text2 = divs[1].text.split('\n') # split by line
                        temp_text2 = filter(None, temp_text2)# remove empty
                        for J in range(0,len(temp_text2)):
                            if 'Price' in temp_text[J]:
                                myPanda.loc[len(legoSets)-1,'selloutDate'] =  pandas.to_datetime(temp_text2[J+1].lstrip())

        # get 3rd party prices
        if '3rd' in div.h3.text:
            temp_text = div.text.split('\n') # split by line
            temp_text = filter(None, temp_text)# remove empty 
            for I in range(0,len(temp_text)):
                if 'Highest' in temp_text[I]:
                    myPanda.loc[len(legoSets)-1,'highPrice_3'] = float(temp_text[I+1].replace("$", ""))
                if 'Lowest' in temp_text[I]:
                    myPanda.loc[len(legoSets)-1,'lowPrice_3'] = float(temp_text[I+1].replace("$", ""))
                if 'Average' in temp_text[I]:
                    myPanda.loc[len(legoSets)-1,'avgPrice_3'] = float(temp_text[I+1].replace("$", ""))
                if 'Current' in temp_text[I]:
                    try: 
                        myPanda.loc[len(legoSets)-1,'currentPrice_3'] = float(temp_text[I+1].replace("$", ""))
                    except ValueError:
                        myPanda.loc[len(legoSets)-1,'currentPrice_3'] = 'NaN' # means not in stock
    return myPanda


## Brickset Scraping Function

In [16]:
# brickset search
p_dd = re.compile(ur'\d.\d')
p_nn = re.compile(ur'^[0-9]*')

def getBrickSets(SetNumber,myPanda):
    tryurl =  "http://brickset.com/sets/"+SetNumber # setup the url
    # print tryurl
    response = urllib2.urlopen(tryurl)
    html = response.read()
    
    parsedHtml = BeautifulSoup(html,'lxml')  # parse with BeautifulSoup
    myPanda.loc[len(legoSets)-1,'Theme']=str(parsedHtml.find("dt",text="Theme").find_next_sibling("dd").text)
    myPanda.loc[len(legoSets)-1,'introYear']=str(parsedHtml.find("dt",text="Year released").find_next_sibling("dd").text)
    myPanda.loc[len(legoSets)-1,'pieces']=str(parsedHtml.find("dt",text="Pieces").find_next_sibling("dd").text)
    usd = parsedHtml.find("dt",text="RRP").find_next_sibling("dd").text.find('$')
    myPanda.loc[len(legoSets)-1,'retailPrice']=parsedHtml.find("dt",text="RRP").find_next_sibling("dd").text[usd+1:]
    try:
        myPanda.loc[len(legoSets)-1,'minifigs']=int(parsedHtml.find("dt",text="Minifigs").find_next_sibling("dd").text)
    except:
        pass
    
    # brickset rating
    try: 
        text = str(parsedHtml.find_all("div", {"class":"rating"})[0])
        myPanda.loc[len(legoSets)-1,'rating_B']=float(re.search(p_dd,text).group(0))
    except:
        pass
    
    # how many want, how many have it
    for item in parsedHtml.find_all("li"):
        if " people own this set" in item.text:
            myPanda.loc[len(legoSets)-1,'have_it']=re.search(p_nn,item.text).group(0)
        if " want this set" in item.text:
            myPanda.loc[len(legoSets)-1,'want_it']=re.search(p_nn,item.text).group(0)
        
    return myPanda
    # API: get the specific set and process it
    # first search for the set number
    #Lsets = brickset_client.service.getSets(brickset_client.API_KEY,'sharathbennur',SetNumber)
    # now use the set number we got back to get everything else about the set
    #Lset = brickset_client.service.getSet(brickset_client.API_KEY, 'sharathbennur',setID_BrickSet)


In [17]:
# first time
legoSets = getCamels(ASIN, legoSets)

In [18]:
legoSets = getBrickSets(SetNumber,legoSets)
legoSets

,SetNumber,SetName,Theme,ASIN,retailPrice,currentPrice_A,highPrice_A,lowPrice_A,avgPrice_A,highPrice_3,...,introDate,introYear,selloutDate,highPriceDate_A,pieces,minifigs,rating_B,want_it,have_it,currentPrice_3
0,75102,LEGO Star Wars Poe's X-Wing Fighter 75102 Buil...,Star Wars,B00WHZ6WOO,79.99,79,NaN,NaN,NaN,112.20,...,2015-09-04,2015,NaN,NaN,717,NaN,NaN,1321,478,89.99
1,79007,LEGO LOTR Battle at The Black Gate 79007,The Lord of the Rings,B00C9X59UM,59.99 / 69.99€,NaN,59.99,41.12,49.46,68.12,...,2013-05-25,2013,2015-01-06 23:39:00,2015-01-06 00:00:00,656,5,4.7,2128,4245,NaN


## SQL DB

In [13]:
########### SQL DB SETUP ############

# 'SetNumber','SetName','Theme','ASIN','retailPrice','currentPrice_A',
#     'highPrice_A','lowPrice_A','avgPrice_A','highPrice_3','lowPrice_3','avgPrice_3',
#     'introDate','introYear','selloutDate','highPriceDate_A','pieces','minifigs','rating_B',
#        'want_it','have_it'

# CREATE TABLE legoSets 
# (SetNumber VARCHAR(10), 
# SetName TEXT, 
# Theme VARCHAR(25), 
# ASIN VARCHAR(25), 
# retailPrice FLOAT(8,2),
# highPrice_A FLOAT(8,2),
# lowPrice_A FLOAT(8,2),
# avgPrice_A FLOAT(8,2),
# highPrice_3 FLOAT(8,2),
# lowPrice_3 FLOAT(8,2),
# avgPrice_3 FLOAT(8,2),
# introDate  TEXT,
# introYear TEXT,
# selloutDate TEXT,
# highPriceDate_A TEXT,
# pieces SMALLINT,
# minifigs TINYINT,
# rating_B TINYINT,
# want_it SMALLINT,
# have_it SMALLINT);

# CREATE TABLE Themes (venue_id INT(12),  venue_name LONGTEXT, category MEDIUMTEXT, fsq_url MEDIUMTEXT, address MEDIUMTEXT, state TEXT, city TEXT, latitude DOUBLE(12,8) NOT NULL, longitude DOUBLE(12,8) NOT NULL, PRIMARY KEY (venue_id));
# CREATE TABLE untap_beer (beer_id INT(12), beer_name MEDIUMTEXT, brewery_name MEDIUMTEXT, brewery_id INT(12), brewery_slug MEDIUMTEXT, style MEDIUMTEXT, abv FLOAT(3,1), PRIMARY KEY (beer_ID))

#con = mdb.connect('localhost', 'root', '', 'legoSet',charset='utf8')
#legoSets.to_sql(con=con, name='legoSet', if_exists='replace', flavor='mysql', index=False)
# legoSets.to_sql(con=con, name='legoSet', if_exists='replace', flavor='mysql', index=False)
# df = pandas.read_sql('SELECT * FROM legoSet', con)
# df
# # test section to try sql
# df = pandas.read_sql('SELECT * FROM legoSet', con, index_col=None)
# df

In [19]:
fn = '/Users/sharathbennur/Insights/Project/legoSet.pkl'
legoSets.to_pickle(fn)  # where to save it, usually as a .pkl

## Then you can load it back using:
# df = pd.load(fn)

In [20]:
# new row
name = 'LEGO LOTR Battle at The Black Gate 79007'
ASIN = am_search(name) # get ASIN
# Add a new row to the data
legoSets.loc[len(legoSets)] = np.nan
legoSets = amLookupASIN(ASIN,legoSets) # fill in values from amazon
SetNumber=legoSets.loc[len(legoSets)-1,'SetNumber'] # get set number

# fill in from camelcamelcamel
legoSets = getCamels(ASIN, legoSets)
# fill in from brickset
legoSets = getBrickSets(SetNumber,legoSets)
# fill in from ebay
legoSets

,SetNumber,SetName,Theme,ASIN,retailPrice,currentPrice_A,highPrice_A,lowPrice_A,avgPrice_A,highPrice_3,...,introDate,introYear,selloutDate,highPriceDate_A,pieces,minifigs,rating_B,want_it,have_it,currentPrice_3
0,75102,LEGO Star Wars Poe's X-Wing Fighter 75102 Buil...,Star Wars,B00WHZ6WOO,79.99,79,NaN,NaN,NaN,112.20,...,2015-09-04,2015,NaN,NaN,717,NaN,NaN,1321,478,89.99
1,79007,LEGO LOTR Battle at The Black Gate 79007,The Lord of the Rings,B00C9X59UM,59.99 / 69.99€,NaN,59.99,41.12,49.46,68.12,...,2013-05-25,2013,2015-01-06 23:39:00,2015-01-06 00:00:00,656,5,4.7,2128,4245,NaN
2,79007,LEGO LOTR Battle at The Black Gate 79007,The Lord of the Rings,B00C9X59UM,59.99 / 69.99€,NaN,59.99,41.12,49.46,68.12,...,2013-05-25,2013,2015-01-06 23:39:00,2015-01-06 00:00:00,656,5,4.7,2128,4245,NaN


## Brickset Theme Scratch Pad 

In [ ]:
Themes = ['Advanced-Models','Adventurers','Aquazone','Architecture','Atlantis','Baby','Basic',
         'Batman','Belville','Boats','Bionicle','Cars','Castle','City','Creator','DC-Comics-Super-Heroes',
         'Dino','Disney-Princess','Duplo','Education','Elves','Explore','Friends','Games','Harry-Potter',
          'HERO-Factory','Ideas','Indiana-Jones','Island-Xtreme-Stunts','Jack-Stone','Juniors','Jurassic-World',
         'Legends-of-Chima','LEGOLAND','Make-and-Create','Marvel-Super-Heroes','Master-Builder-Academy',
         'Mindstorms','Minecraft','Mixels','Monster-Fighters','Ninjago','Pharaohs-Quest','Pirates',
          'Pirates-of-the-Caribbean','Power-Miners','Power-Functions','Prince-of-Persia','Quatro',
         'Racers','Rock-Raiders','Scooby-Doo','Seasonal','Serious-Play','Space','Speed-Champions','Spider-Man',
         'SpongeBob-SquarePants','Sports','Sports','Star-Wars','Studios','Technic','Teenage-Mutant-Ninja-Turtles',
         'The-Hobbit','The-LEGO-Movie','The-Lone-Ranger','The-Lord-of-the-Rings','The-Simpsons','Town',
         'Toy-Story','Trains','Ultra-Agents','Vikings','World-City','World-Racers']

Theme = 'Star-Wars'
getMore = True
N=0
p_nn = re.compile(ur'^[0-9]*')
while getMore:
    N=N+1; # increment page number
    # continue or not?
    tryurl =  "http://brickset.com/sets/theme-"+Theme+"/page-"+str(N) # setup the url
    print tryurl
    response = urllib2.urlopen(tryurl)
    html = response.read()
    parsedHtml = BeautifulSoup(html,'lxml')  # parse with BeautifulSoup
    # make sure page is valid
    items = parsedHtml.find_all("article", {"class":"set"}) # individual set name
    if items:
        divs = parsedHtml.find_all("div", {"class":"meta"}) # individual set name
        for div in divs: # for each item found
            print div.a.text
            SN = re.search(p_nn,div.a.text).group(0) # get set number
            if SN not in str(legoSets.SetNumber): # if we don't have the set, get it
                try:
                    time.sleep(5) 
                    name = div.a.text
                    ASIN = am_search(name) # get ASIN
                    print ASIN
                    if (isinstance(ASIN, str)) or (ASIN is not None): 
                        # if its valid, add a new line
                        legoSets.loc[len(legoSets)] = np.nan
                        legoSets = amLookupASIN(ASIN,legoSets)
                        SetNumber=legoSets.loc[len(legoSets)-1,'SetNumber']
                        # fill in from camelcamelcamel
                        legoSets = getCamels(ASIN, legoSets)
                        # fill in from brickset
                        legoSets = getBrickSets(SetNumber,legoSets)
                except: # if it doesnt work, move to the next item
                    pass
    else: # if no more valid pages
        getMore = False

fn = '/Users/sharathbennur/Insights/Project/legoSet.pkl'
legoSets.to_pickle(fn)  # where to save it, usually as a .pkl
    
# print parsedHtml.find("dt",text="Theme").find_next_sibling("dd").text
# print parsedHtml.find("dt",text="Year released").find_next_sibling("dd").text
# print parsedHtml.find("dt",text="Pieces").find_next_sibling("dd").text
# usd = parsedHtml.find("dt",text="RRP").find_next_sibling("dd").text.find('$')
# print parsedHtml.find("dt",text="RRP").find_next_sibling("dd").text[usd+1:]


# tryurl =  "http://brickset.com/sets/"+SetNumber # setup the url
# print tryurl
# response = urllib2.urlopen(tryurl)
# html = response.read()
# parsedHtml = BeautifulSoup(html,'lxml')  # parse with BeautifulSoup

# print parsedHtml.find("dt",text="Theme").find_next_sibling("dd").text
# print parsedHtml.find("dt",text="Year released").find_next_sibling("dd").text
# print parsedHtml.find("dt",text="Pieces").find_next_sibling("dd").text
# usd = parsedHtml.find("dt",text="RRP").find_next_sibling("dd").text.find('$')
# print parsedHtml.find("dt",text="RRP").find_next_sibling("dd").text[usd+1:]

In [ ]:
fn = '/Users/sharathbennur/Insights/Project/legoSet.pkl'
legoSets.to_pickle(fn)  # where to save it, usually as a .pkl
legoSets

## C3 Scratch Pad Code

In [ ]:
ASIN = 'B00C9X59UM'
tryurl =  "http://www.camelcamelcamel.com/product/"+ASIN # setup the url
# print tryurl
#response = urllib2.urlopen(tryurl)
#html = response.read()

parsedHtml = BeautifulSoup(html,"lxml")  # parse with BeautifulSoup
items = parsedHtml.find_all(attrs={"class": "smalltext grey"})
for item in items:
    if 'since' in item.text:
        print pandas.to_datetime(item.text.lstrip().replace("* since", "").replace('+ of the last 50 price changes',''))
        #myPanda.loc[len(legoSets)-1,'introDate'] =  pandas.to_datetime(item.text.lstrip)
# divs = parsedHtml.find_all("div", {"class":"yui3-u-1-2"}) # find the divs with the price information
# temp_text = divs[0].text.split('\n') # split by line  # print div.text
# temp_text = filter(None, temp_text) # remove empty
# print temp_text
# for J in range(0,len(temp_text)): 
#    if 'Highest' in temp_text[J]:             # get the last time it was in stock
#        print  time.strptime(temp_text[J+2].lstrip(), "%b %d, %Y")


## Brickset Scraper Scratch Pad 

In [ ]:
# SetNumber
# tryurl =  "http://brickset.com/sets/"+SetNumber # setup the url
# # print tryurl
# response = urllib2.urlopen(tryurl)
# html = response.read()
p_dd = re.compile(ur'\d.\d')
p_nn = re.compile(ur'^[0-9]*')
parsedHtml = BeautifulSoup(html,'lxml')  # parse with BeautifulSoup
# print parsedHtml.find("dt",text="Theme").find_next_sibling("dd").text
# text = str(parsedHtml.find_all("div", {"class":"rating"})[0])
# print float(re.search(p_dd,text).group(0))
for item in parsedHtml.find_all("li"):
    if " people own this set" in item.text:
        print re.search(p_nn,item.text).group(0)
    if " want this set" in item.text:
        print re.search(p_nn,item.text).group(0)
    

# parsedHtml.find('rating').a.text
# print parsedHtml.find("dt",text="Theme").find_next_sibling("dd").text
# print parsedHtml.find("dt",text="Year released").find_next_sibling("dd").text
# print parsedHtml.find("dt",text="Pieces").find_next_sibling("dd").text
# usd = parsedHtml.find("dt",text="RRP").find_next_sibling("dd").text.find('$')
# print parsedHtml.find("dt",text="RRP").find_next_sibling("dd").text[usd+1:]


# tryurl =  "http://brickset.com/sets/"+SetNumber # setup the url
# print tryurl
# response = urllib2.urlopen(tryurl)
# html = response.read()
# parsedHtml = BeautifulSoup(html,'lxml')  # parse with BeautifulSoup

# print parsedHtml.find("dt",text="Theme").find_next_sibling("dd").text
# print parsedHtml.find("dt",text="Year released").find_next_sibling("dd").text
# print parsedHtml.find("dt",text="Pieces").find_next_sibling("dd").text
# usd = parsedHtml.find("dt",text="RRP").find_next_sibling("dd").text.find('$')
# print parsedHtml.find("dt",text="RRP").find_next_sibling("dd").text[usd+1:]

## Ebay Scratch Pad

In [ ]:
import datetime
from ebaysdk.finding import Connection

api = Connection(appid='SubhashB-f237-485e-9a38-318aa7e72eee')
response = api.execute('findItemsAdvanced', {'keywords': 'legos'})

assert(response.reply.ack == 'Success')
assert(type(response.reply.timestamp) == datetime.datetime)
assert(type(response.reply.searchResult.item) == list)

item = response.reply.searchResult.item[0]
assert(type(item.listingInfo.endTime) == datetime.datetime)
assert(type(response.dict()) == dict)

## To get data from SQL database

In [ ]:
# make connection
conn = mdb.connect(host="localhost", user="root", passwd="", db="legoSets")
# connect
cursor = conn.cursor()
# pick data
cursor.execute('select Name, Continent, Population, LifeExpectancy, GNP from Country');
# get data
rows = cursor.fetchall()
# put in panda
df = pd.DataFrame( [[ij for ij in i] for i in rows] )